In [2]:
import requests
import json
from census import Census
from us import states
from config import api_key
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#API calls census using wrapper
c = Census(api_key, year=2017)
census_data = c.acs5.get(("B01003_001E","B01002_001E"),{'for': 'county:*', 'in': 'state:*' })

#converting to dataFrame
census_pd = pd.DataFrame(census_data)

#Renaming the columns
census_pd=census_pd.rename(columns={"B01003_001E":"Total Population","B01002_001E":"Median Age","state":"fips"})
census_pd.head()


,Total Population,Median Age,fips,county
0,34933.0,38.1,72,047
1,11297.0,43.5,72,095
2,21661.0,36.4,72,111
3,148863.0,39.5,72,113
4,38970.0,42.0,72,131


In [4]:
Population=census_pd.groupby('fips')['Total Population'].sum()
final_fips=census_pd.groupby('fips')['fips'].first()

state_population=pd.DataFrame({"Total Population":Population,"State Fips":final_fips})
state_population.loc[:,'State Fips']=state_population["State Fips"].astype('int64')

state_population.head()

,Total Population,State Fips
fips,,
01,4850771.0,1
02,738565.0,2
04,6809946.0,4
05,2977944.0,5
06,38982847.0,6


In [5]:
zip_path="state_fips_master.csv"
zip_df=pd.read_csv(zip_path)
zip_df=zip_df.rename(columns={"fips":"State Fips"})
zip_df.head()

,state_name,state_abbr,long_name,State Fips,sumlev,region,division,state,region_name,division_name
0,Alabama,AL,Alabama AL,1,40,3,6,1,South,East South Central
1,Alaska,AK,Alaska AK,2,40,4,9,2,West,Pacific
2,Arizona,AZ,Arizona AZ,4,40,4,8,4,West,Mountain
3,Arkansas,AR,Arkansas AR,5,40,3,7,5,South,West South Central
4,California,CA,California CA,6,40,4,9,6,West,Pacific


In [6]:
#Merging the census_pd with state_zip_df to obtain state_name column in Census dataFrame

Census_df=pd.merge(state_population,zip_df,on="State Fips")
Census_df=Census_df.rename(columns={"State Fips":"fips","state_name":"State Name"})
Census_df.head()

,Total Population,fips,State Name,state_abbr,long_name,sumlev,region,division,state,region_name,division_name
0,4850771.0,1,Alabama,AL,Alabama AL,40,3,6,1,South,East South Central
1,738565.0,2,Alaska,AK,Alaska AK,40,4,9,2,West,Pacific
2,6809946.0,4,Arizona,AZ,Arizona AZ,40,4,8,4,West,Mountain
3,2977944.0,5,Arkansas,AR,Arkansas AR,40,3,7,5,South,West South Central
4,38982847.0,6,California,CA,California CA,40,4,9,6,West,Pacific


In [7]:
csv_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

# Read the CSV into a Pandas DataFrame
covid_df = pd.read_csv(csv_path)

# Print the first five rows of data to the screen
covid_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [8]:
#Covid-19 Data on County level

#extracting latest information from the live data
cases_covid=covid_df.groupby(['fips','county','state'])['cases'].last()
deaths_covid=covid_df.groupby(['fips','county','state'])['deaths'].last()
date_covid=covid_df.groupby(['fips','county','state'])['date'].last()
fips_covid=covid_df.groupby(['fips','county','state'])['fips'].first()
state_covid=covid_df.groupby(['fips','county','state'])['state'].first()
county_covid=covid_df.groupby(['fips','county','state'])['county'].first()

#creating new dataframe
County_Covid_df=pd.DataFrame({"Date":date_covid,
                             "County":county_covid,
                             "Total Number of Covid-19 Cases":cases_covid,
                             "Death due to Covid-19":deaths_covid})
County_Covid_df.head()

,,,Date,County,Total Number of Covid-19 Cases,Death due to Covid-19
fips,county,state,,,,
1001.0,Autauga,Alabama,2020-04-10,Autauga,17,1
1003.0,Baldwin,Alabama,2020-04-10,Baldwin,59,1
1005.0,Barbour,Alabama,2020-04-10,Barbour,9,0
1007.0,Bibb,Alabama,2020-04-10,Bibb,11,0
1009.0,Blount,Alabama,2020-04-10,Blount,12,0


In [9]:
#Covid-19 Data on State Level

cases_state=covid_df.groupby(['state'])['cases'].sum()
deaths_state=covid_df.groupby(['state'])['deaths'].sum()
date_state=covid_df.groupby(['state'])['date'].last()
state_name=covid_df.groupby(['state'])['state'].last()

State_Covid_df=pd.DataFrame({"Date":date_state,"State Name":state_name,"Total Covid-19 Cases":cases_state,"Total Deaths Covid-19":deaths_state})
State_Covid_df.head()

,Date,State Name,Total Covid-19 Cases,Total Deaths Covid-19
state,,,,
Alabama,2020-04-10,Alabama,26064,571
Alaska,2020-04-10,Alaska,2753,48
Arizona,2020-04-10,Arizona,29834,705
Arkansas,2020-04-10,Arkansas,13472,216
California,2020-04-10,California,212826,5007


In [20]:
complete_df=pd.merge(State_Covid_df,Census_df,on='State Name')
complete_df=complete_df[["Date","State Name","Total Covid-19 Cases","Total Deaths Covid-19","Total Population","fips"]]
complete_df

,Date,State Name,Total Covid-19 Cases,Total Deaths Covid-19,Total Population,fips
0,2020-04-10,Alabama,26064,571,4850771.0,1
1,2020-04-10,Alaska,2753,48,738565.0,2
2,2020-04-10,Arizona,29834,705,6809946.0,4
3,2020-04-10,Arkansas,13472,216,2977944.0,5
4,2020-04-10,California,212826,5007,38982847.0,6
5,2020-04-10,Colorado,68403,1892,5436519.0,8
6,2020-04-10,Connecticut,81643,2613,3594478.0,9
7,2020-04-10,Delaware,9726,201,943732.0,10
8,2020-04-10,Florida,165335,2974,20278447.0,12
9,2020-04-10,Georgia,101105,3506,10201635.0,13


In [26]:
#Adding Latitude and Longitude for HeatMap

latlng_path="statelatlong.csv"
latlng_df=pd.read_csv(latlng_path)
latlng_df=latlng_df.rename(columns={"City":"State Name"})
HeatMap_df=pd.merge(latlng_df,complete_df,on="State Name")
HeatMap_df.head()

,State,Latitude,Longitude,State Name,Date,Total Covid-19 Cases,Total Deaths Covid-19,Total Population,fips
0,AL,32.601011,-86.680736,Alabama,2020-04-10,26064,571,4850771.0,1
1,AK,61.302501,-158.775020,Alaska,2020-04-10,2753,48,738565.0,2
2,AZ,34.168219,-111.930907,Arizona,2020-04-10,29834,705,6809946.0,4
3,AR,34.751928,-92.131378,Arkansas,2020-04-10,13472,216,2977944.0,5
4,CA,37.271875,-119.270415,California,2020-04-10,212826,5007,38982847.0,6


In [ ]:
locations = HeatMap_df[["Lat", "Lng"]]